# Extracción y preparación de datos.  

En esta sección extraeremos los datos desde nuestro archivo fuente que es un json, al cual le aplicaremos los cambios necesarios para quedarnos solamente con la infromación que se nos explico es útil para nuestro modelo predictivo. 

In [1]:
import json
import pandas as pd
import numpy as np# Función para preprocesamiento
def preprocess_text(text):
    text = re.sub(r"XX|xx", "", text)  # Eliminar "XX"
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Mantener solo letras
    text = text.lower()  # Convertir a minúsculas
    tokens = text.split()  # Tokenizar
    tokens = [word for word in tokens if word not in stop_words]  # Eliminar stopwords
    return " ".join(tokens)

In [2]:
with open('/home/melanie/PycharmProjects/final-exam-pcd2024-autumn/raw_data/tickets_classification_eng.json', 'r') as file:
    datos = json.load(file)

In [3]:
df = pd.json_normalize(datos)
print(f"The dimension of the dataframe are:", df.shape)
df.head(1)

The dimension of the dataframe are: (78313, 22)


,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
0,complaint-public-v2,complaint,3211475,0.0,None,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,None,Credit card debt,Yes,,Debt is not yours,Consent not provided


Podemos observar que tenemos bastantes columnas que no son necesarias, por lo que solamente nos quedaremos con las columnas que nos fueron indicadas y haremos los cambios indicados a ellas, tales como su selección, cambio de nombre, etc. 

In [4]:
df = df[['_source.complaint_what_happened', '_source.product', '_source.sub_product']]
df.head(2)

,_source.complaint_what_happened,_source.product,_source.sub_product
0,,Debt collection,Credit card debt
1,Good morning my name is XXXX XXXX and I apprec...,Debt collection,Credit card debt


In [5]:
df.rename(columns={"_source.complaint_what_happened": "complaint_what_happened", "_source.product": "category",
                   "_source.sub_product": "sub_product"}, inplace=True)
df.head(2)

,complaint_what_happened,category,sub_product
0,,Debt collection,Credit card debt
1,Good morning my name is XXXX XXXX and I apprec...,Debt collection,Credit card debt


Crearemos una nueva columna donde uniremos la categoría y el subproducto para que estén juntos y eliminaremos las columnas de donde se obtuvo la información para eliminar redundancia. 

In [6]:
df['ticket_classification'] = df['category'] + ' + ' + df['sub_product']

df.drop(columns=['category', 'sub_product'], inplace=True)

df.head(2)

,complaint_what_happened,ticket_classification
0,,Debt collection + Credit card debt
1,Good morning my name is XXXX XXXX and I apprec...,Debt collection + Credit card debt


# Limpieza de datos. 

Haremos una visualización de nuestros datos para observar su granularidad y eliminaremos los valores nulos que tengamos en nuestro df, posteriormente guardaremos nuestro nuevos datos para comenzar con el análisis y modelado. 

In [7]:
def reporte(df):
    dtyp = pd.DataFrame(df.dtypes, columns=['Tipo'])
    missing = pd.DataFrame(df.isnull().sum(), columns=['Valores_Nulos'])
    unival = pd.DataFrame(df.nunique(), columns=['Valores_Unicos'])
    return dtyp.join(missing).join(unival)#.join(maximo).join(minimo)
reporte(df)

,Tipo,Valores_Nulos,Valores_Unicos
complaint_what_happened,object,0,20931
ticket_classification,object,10571,87


In [8]:
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [10]:
reporte(df)

,Tipo,Valores_Nulos,Valores_Unicos
complaint_what_happened,object,57241,20930
ticket_classification,object,10571,87


In [11]:
df.dropna(inplace=True)

In [12]:
reporte(df)

,Tipo,Valores_Nulos,Valores_Unicos
complaint_what_happened,object,0,18824
ticket_classification,object,0,78


In [13]:
# Reiniciar índice
df.reset_index(inplace=True, drop=True)

In [16]:
df.to_csv('ata.csv')

In [17]:
df

,complaint_what_happened,ticket_classification
0,Good morning my name is XXXX XXXX and I apprec...,Debt collection + Credit card debt
1,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Credit card or prepaid card + General-purpose ...
2,Chase Card was reported on XX/XX/2019. However...,"Credit reporting, credit repair services, or o..."
3,"On XX/XX/2018, while trying to book a XXXX XX...","Credit reporting, credit repair services, or o..."
4,my grand son give me check for {$1600.00} i de...,Checking or savings account + Checking account
...,...,...
18958,My husband passed away. Chase bank put check o...,Checking or savings account + Checking account
18959,After being a Chase Card customer for well ove...,Credit card or prepaid card + General-purpose ...
18960,"On Wednesday, XX/XX/XXXX I called Chas, my XXX...",Credit card or prepaid card + General-purpose ...
18961,I am not familiar with XXXX pay and did not un...,Checking or savings account + Checking account


In [ ]:
df.r